In [59]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.engine import training
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Average, LSTM
from keras.layers.convolutional import Convolution1D, AveragePooling1D, MaxPooling1D
from keras.losses import categorical_crossentropy
from keras.models import Model, Input, Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.python.framework.ops import Tensor
from typing import Tuple, List
import glob
import numpy as np
import pandas as pd
import os

In [2]:
def onehot_encoding(target_32bp, target_20bp):
    # one-hot encoding ACGT_32bp

    ACGT = [['A'], ['C'], ['G'], ['T']]

    onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

    onehot_encoder.fit(ACGT)

    RNA_encoded_32bp = []

    RNA_encoded_20bp = []

    target_32bp = np.array(target_32bp)

    target_20bp = np.array(target_20bp)

    for i in range(len(target_32bp)):
        target_32bp_raw = np.array(list(target_32bp[i])).reshape(-1, 1)

        target_32bp_encoded = onehot_encoder.transform(target_32bp_raw)

        target_20bp_raw = np.array(list(target_20bp[i])).reshape(-1, 1)

        target_20bp_encoded = onehot_encoder.transform(target_20bp_raw)

        RNA_encoded_32bp.append(target_32bp_encoded)

        RNA_encoded_20bp.append(target_20bp_encoded)

    return RNA_encoded_32bp, RNA_encoded_20bp

In [3]:
RNA_train = pd.read_csv("./RNA_train.csv")
RNA_train.describe(include="all")
RNA_test = pd.read_csv("./RNA_test.csv")
RNA_test.describe(include="all")

train_target_50bp = RNA_train.target_50bp
train_target_34bp = RNA_train.target_34bp
train_target_20bp = RNA_train.target_20bp
train_indel = RNA_train.indel

test_target_50bp = RNA_test.target_50bp
test_target_34bp = RNA_test.target_34bp
test_target_20bp = RNA_test.target_20bp
test_indel = RNA_test.indel

In [4]:
train_target_encoded_32bp, train_target_encoded_20bp = onehot_encoding(train_target_34bp, train_target_20bp)
test_target_encoded_32bp, test_target_encoded_20bp = onehot_encoding(test_target_34bp, test_target_20bp)

In [61]:
x_train = train_target_encoded_32bp
x_train = np.array(x_train)

y_train = train_indel
y_train = list(y_train)
y_train = np.reshape(list(y_train),(-1,1)) 

X_train, X_val, Y_train, Y_val = train_test_split (x_train,y_train,test_size=0.2,random_state=123)

X_train = np.array(X_train)
X_val = np.array(X_val)
Y_train = np.array(Y_train)
Y_val = np.array(Y_val)


x_test = test_target_encoded_32bp
x_test = np.array(x_test)

y_test = test_indel
y_test2 = list(y_test)
y_test = np.reshape(list(y_test2),(-1,1))


print(y_train)

[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]


In [6]:
batch_size = 32
epochs = 300

In [12]:
input_img = Input(shape=(34, 4))
x = Convolution1D(32, 5, padding='same')(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Convolution1D(16, 5, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Convolution1D(1, 5, padding='same')(x)
x = BatchNormalization()(x)
encoded = Activation('relu')(x)

x = Convolution1D(1, 5, padding='same')(encoded)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Convolution1D(16, 5, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Convolution1D(4, 5, padding='same')(x)
x = BatchNormalization()(x)
decoded = Activation('softmax')(x)

In [13]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 34, 4)             0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 34, 32)            672       
_________________________________________________________________
batch_normalization_19 (Batc (None, 34, 32)            128       
_________________________________________________________________
activation_19 (Activation)   (None, 34, 32)            0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 34, 16)            2576      
_________________________________________________________________
batch_normalization_20 (Batc (None, 34, 16)            64        
_________________________________________________________________
activation_20 (Activation)   (None, 34, 16)            0         
__________

In [14]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = autoencoder.fit(X_train, X_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_val, X_val),
                    callbacks=[early_stopping],
                    shuffle=True)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 12000 samples, validate on 3000 samples
Epoch 1/300
12000/12000 [==============================] - 9s 782us/step - loss: 0.3116 - val_loss: 0.2219
Epoch 2/300
12000/12000 [==============================] - 4s 298us/step - loss: 0.1032 - val_loss: 0.0772
Epoch 3/300
12000/12000 [==============================] - 4s 353us/step - loss: 0.0408 - val_loss: 0.0344
Epoch 4/300
12000/12000 [==============================] - 4s 335us/step - loss: 0.0308 - val_loss: 0.0280
Epoch 5/300
12000/12000 [==============================] - 4s 324us/step - loss: 0.0259 - val_loss: 0.0234
Epoch 6/300
12000/12000 [==============================] - 4s 323us/step - loss: 0.0228 - val_loss: 0.0212
Epoch 7/300
12000/12000 [==============================] - 4s 326us/step - loss: 0.0207 - val_loss: 0.0194
Epoch 8/300
12000/12000 [==============================] - 4s 328us/step - l

In [43]:
decoded_squence = autoencoder.predict(x_test)
ary_decoded = decoded_squence
ary_test = x_test
print(decoded_squence)

[[[6.1624417e-09 6.3294691e-12 1.0000000e+00 4.8972091e-14]
  [9.9999928e-01 4.9282767e-07 1.8135848e-08 2.0014539e-07]
  [8.8406099e-22 1.0000000e+00 2.0085832e-15 9.5419889e-17]
  ...
  [3.6434391e-20 1.0000000e+00 4.2391399e-12 4.0327781e-14]
  [1.0000000e+00 1.3379012e-10 1.4376600e-10 3.6421220e-11]
  [3.3130631e-04 9.9963307e-01 3.7467622e-14 3.5656001e-05]]

 [[1.5722836e-09 1.0000000e+00 5.4992821e-13 2.8807980e-15]
  [3.4585090e-14 6.5604721e-13 4.1952860e-12 1.0000000e+00]
  [2.9497202e-16 2.5515237e-16 1.0000000e+00 1.9353614e-14]
  ...
  [2.8841376e-12 3.6873843e-08 9.9999738e-01 2.6380385e-06]
  [1.3539264e-23 1.0000000e+00 9.6894779e-25 2.5163812e-21]
  [9.9993145e-01 2.2072207e-05 4.6343077e-05 1.7094364e-07]]

 [[3.3310639e-07 3.0333049e-06 1.1926802e-08 9.9999666e-01]
  [4.4888970e-13 5.3099199e-11 1.0000000e+00 1.2034337e-11]
  [4.1255608e-15 1.0000000e+00 6.1743481e-09 6.1563563e-09]
  ...
  [9.1449740e-28 1.0000000e+00 1.6922856e-17 1.4569442e-18]
  [2.1491772e-19 1

In [18]:
len = ary_decoded.shape[0]
print(len)
sequence_len = ary_decoded.shape[1]
print(sequence_len)

1292
34


In [19]:
for i in range(len):
    for j in range(sequence_len):
        maxx = np.where(max(ary_decoded[i][j]) == ary_decoded[i][j])[0][0]
        for k in range(4):
            if k==maxx :
                ary_decoded[i][j][k]=1
            else :
                ary_decoded[i][j][k]=0

ary_decoded = np.reshape(ary_decoded, (-1))
ary_test = np.reshape(x_test, (-1))
corrr = np.corrcoef(ary_decoded, ary_test)
print(corrr[0][1])

0.9987858920658048


In [20]:
cnt = 0
for i in range(1292) :
    if ary_decoded[i]!=ary_test[i] :
        cnt = cnt + 1
        
print(cnt)

0


In [78]:
encoder = Model(input_img, encoded)
encoder.save('encoder_34.h5')

In [57]:
encoded_train = encoder.predict(X_train)
encoded_val = encoder.predict(X_val)
encoded_test = encoder.predict(x_test)


In [66]:
model = Sequential()
model.add(LSTM(128, input_shape=(34,1,)))
model.add(Dropout(0,2))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________


In [75]:
early_stopping = EarlyStopping(patience = 10)

model.fit(encoded_train, Y_train, epochs=100, batch_size=128, validation_data=(encoded_val, Y_val), callbacks=[early_stopping])


Train on 12000 samples, validate on 3000 samples
Epoch 1/100
12000/12000 [==============================] - 7s 570us/step - loss: 0.0590 - val_loss: 0.0586
Epoch 2/100
12000/12000 [==============================] - 7s 543us/step - loss: 0.0587 - val_loss: 0.0613
Epoch 3/100
12000/12000 [==============================] - 7s 550us/step - loss: 0.0587 - val_loss: 0.0591
Epoch 4/100
12000/12000 [==============================] - 7s 553us/step - loss: 0.0582 - val_loss: 0.0625
Epoch 5/100
12000/12000 [==============================] - 7s 555us/step - loss: 0.0586 - val_loss: 0.0570
Epoch 6/100
12000/12000 [==============================] - 7s 574us/step - loss: 0.0579 - val_loss: 0.0568
Epoch 7/100
12000/12000 [==============================] - 7s 578us/step - loss: 0.0571 - val_loss: 0.0562
Epoch 8/100
12000/12000 [==============================] - 7s 588us/step - loss: 0.0565 - val_loss: 0.0568
Epoch 9/100
12000/12000 [==============================] - 7s 603us/step - loss: 0.0569 - val_l

In [76]:
predict_model = model.predict(encoded_test, batch_size=50, verbose=0)
predict_model = predict_model.flatten()

print(predict_model)

[ 0.1265777  -0.00561314  0.17285797 ...  0.6206628   0.5707806
  0.65318793]


In [77]:
list_y_test = y_test2
list_y_predict = predict_model

com_list = np.stack([list_y_test, list_y_predict])

df = pd.DataFrame(com_list).T
corr = df.corr(method = 'spearman')
print(corr)

          0         1
0  1.000000  0.693779
1  0.693779  1.000000
